## Import Required Libraries

In [76]:
import pandas as pd
import openai
import uuid
import json
import numpy as np

# This is a helper function for easy input and access to openai's API
https://github.com/shreyashankar/gpt3-sandbox

In [77]:
def set_openai_key(key):
    openai.api_key = key
    
class Example:
    def __init__(self, inp, out):
        self.input = inp
        self.output = out
        self.id = uuid.uuid4().hex

    def get_input(self):
        return self.input

    def get_output(self):
        return self.output

    def get_id(self):
        return self.id

    def as_dict(self):
        return {
            "input": self.get_input(),
            "output": self.get_output(),
            "id": self.get_id(),
        }


class GPT:
    def __init__(self,
                 engine='davinci',
                 temperature=0.5,
                 max_tokens=100,
                 input_prefix="input: ",
                 input_suffix="\n",
                 output_prefix="",
                 output_suffix="",
                 append_output_prefix_to_query=False):
        self.examples = {}
        self.engine = engine
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.input_prefix = input_prefix
        self.input_suffix = input_suffix
        self.output_prefix = output_prefix
        self.output_suffix = output_suffix
        self.append_output_prefix_to_query = append_output_prefix_to_query
        self.stop = (output_suffix + input_prefix).strip()

    def add_example(self, ex):
        assert isinstance(ex, Example), "Please create an Example object."
        self.examples[ex.get_id()] = ex

    def delete_example(self, id):
        if id in self.examples:
            del self.examples[id]

    def get_example(self, id):
        return self.examples.get(id, None)

    def get_all_examples(self):
        return {k: v.as_dict() for k, v in self.examples.items()}

    def get_prime_text(self):
        return "".join(
            [self.format_example(ex) for ex in self.examples.values()])

    def get_engine(self):
        return self.engine

    def get_temperature(self):
        return self.temperature

    def get_max_tokens(self):
        return self.max_tokens

    def craft_query(self, prompt):
        q = self.get_prime_text(
        ) + self.input_prefix + prompt + self.input_suffix
        if self.append_output_prefix_to_query:
            q = q + self.output_prefix
        return q

    def submit_request(self, prompt):
        response = openai.Completion.create(engine=self.get_engine(),
                                            prompt=self.craft_query(prompt),
                                            max_tokens=self.get_max_tokens(),
                                            temperature=self.get_temperature(),
                                            top_p=1,
                                            n=1,
                                            stream=False,
                                            stop=self.stop)
        return response

    def get_top_reply(self, prompt):
        response = self.submit_request(prompt)
        return response['choices'][0]['text']

    def format_example(self, ex):
        return self.input_prefix + ex.get_input(
        ) + self.input_suffix + self.output_prefix + ex.get_output(
        ) + self.output_suffix


### Input your openai secret key here

In [78]:
import getpass
city = getpass.getpass('Input your open ai service key:')

Input your open ai service key: ···················································


In [79]:
openai.api_key = city

# So lets now move onto producing jokes
https://www.kaggle.com/datasets/jiriroz/qa-jokes

In [80]:
gpt3 = GPT(engine="davinci", temperature=0.5, max_tokens=100)

In [81]:
jokes = pd.read_csv('jokes.csv')
jokes.head()

,ID,Question,Answer
0,1,Did you hear about the Native American man tha...,He nearly drown in his own tea pee.
1,2,What's the best anti diarrheal prescription?,Mycheexarphlexin
2,3,What do you call a person who is outside a doo...,Matt
3,4,Which Star Trek character is a member of the m...,Jean-Luc Pickacard
4,5,What's the difference between a bullet and a h...,A bullet doesn't miss Harambe


In [82]:
len(jokes)

38269

In [83]:
rand = np.random.randint(0,high=len(jokes),size=1)
jokes['Question'][rand[0]]

"What happens if you violate Reddit's content policy?"

In [84]:
for i in range(5):
    rand = np.random.randint(0,high=len(jokes),size=1)
    ques = jokes['Question'][rand[0]]
    ans = jokes['Answer'][rand[0]]
    print(ques , " : ", ans)
    gpt3.add_example(Example(ques,ans))

What do you get when you put a flight stick in an egg?  :  A yoke. 
What's the difference between the Mafia and the Government?  :  Only one of them is organized.
Why can't you feel photons?  :  Because they are light. 
What do old cars and dead chickens have in common?  :  You'll usually get more money for them if you sell them for parts 
What's Donald's true political party?  :  Whig


We can get individual examples like this:

In [85]:
gpt3.get_example(1)

In [86]:
prompt = "Why do most married men die before their wives?"
output = gpt3.submit_request(prompt)
output.choices[0].text

'They want to. '

In [87]:
prompt = "what did the dog say to a cat?"
output = gpt3.submit_request(prompt)
output.choices[0].text

"I'm not a dog, I'm a cat. "

In [90]:
import torch
torch.save(gpt3, './jokePredictor')

In [100]:
model = torch.load("jokePredictor")
output = model.submit_request("How did pirates communicate before the internet?")
output.choices[0].text

'They used the pirate-net. '

In [101]:
def jokeResponder(txt):
    model = torch.load("jokePredictor")
    output = model.submit_request(txt)
    return output.choices[0].text

In [102]:
def randomJoke():
    rand = np.random.randint(0,high=len(jokes),size=1)
    ques = jokes['Question'][rand[0]]
    model = torch.load("jokePredictor")
    output = model.submit_request(ques)
    return ques+" : "+output.choices[0].text

In [103]:
randomJoke()

"What type of lights were on Noah's Ark? :  Flood lights. "

In [118]:
jokeResponder('What did the vegetable say to the dj?')

'"I\'m a little Brussels sprout." '